# Imports

In [1]:
import math, pyperclip, os, re
from decimal import Decimal
import numpy as np
from functools import reduce
from itertools import product
from datetime import datetime, date, time

# Docs

In [2]:
"""
Data types: Real, Complex, Int, Boolean, String, Undefined
Data structures: Scalar, Vector, Matrix, ...
Variables: v
Functions: fun
Named operators: v operation u
Conversion specifier: 45deg
Data properties: object.property
Oppertation: Numeric, Sets, Comparison 

Data Structures
Tensor: [a, b,, c, d,,, ...]
List: (a, b, c, ...)
Function Body: {expression}

Implicit Operations
2(expression)      *
(expression)2      *
var(expression)    *
fun(parameters)    call

Named operators
on two values: v operation u
on one value: v.operation

Input and Output conversion
1m + 12cm @ cm
2hr + 45min + 1hr + 30min @ datetime
0b1100 * 0xFF @ dec

Basic Operations:
Standard:       | Bitwise: int&bool | Comparison:           | 
+           add | ~             not | ==             equals | 
-      subtract | &&            and | !=          not equal | 
*      multiply | ||             or | <           less than | 
/        divide | <xor>         xor | >        greater then | 
//      int div | <<     left shift | <=      less or equal | 
%       modulus | >>    right shift | >=   greater or equal | 
^         power |                   |                       | 
!     factorial |                   |                       | 
|val|  absolute |                   |                       | 
=    assignment |                   |                       | 

Ternary Operators
= =                return values = function = body
a < x < b          between
a if cond else b   

Higher Ranking Data Structure Operations:
Vector:                 | Matrix:                          | Reductions:
<dot>       dot product | <matmul>   matrix multiplication | <all>
<cross>   cross product | |mat|                            | <any>
|vec|            length | .T              transpose matrix | <
.length          length | 
.angle            angle | 



1 + sqrt 4
1 + sqrt x
1 + $4
1 + 3 root 8
1 + b root x
1 + b $ x

10 nPr 2
n nPr r
10 nCr 2
n nCr r

[1, 2,, 3, 4] # [5, 6,, 7, 8]
[1, 2,, 3, 4].T
mat1.T

5*x

sin 30deg
sin pi
sin2 pi
sini 0.5

Function definitions
fun = x => x^2               single value output
fun = x => [x, x*2, x^2]     tensor outputs
fun = x => (x, x*2, x^2)     multiple outputs
fun = x => {                 piecewise function
    0,   if x < 0 ; 
    x^2, if 0 <= x <= 1 ; 
    x,   else
}
fun(x) = {                  piecewise function
    0   if x < 0
    x^2 if 0 <= x <= 1
    x   else
}

fun(x,y,z) = {
    a = x+2
    b = y*2
    c = z^2
    (a,b,c)
}

if x < 0 {
    x = 0
}

for i = 0:10{
    x += i
}

while x < 0 {
    x += 1
}

@display = scientific 8
@display = hex


Statistical Operations:
"""
None

In [3]:
"""
Conversion specifiers

Time:
    Seconds: s, ms, µs, ns, ps, 
        fs, as, zs, ys, ks, Ms, Gs, Ts, Ps, Es, Zs, Ys
    Other: min, hr, D, W, M, Y, am, pm
    
Distance:
    Metric: m, dm, cm, mm, µm, nm, pm, km, 
        fm, am, zm, ym, Mm, Gm, Tm, Pm, Em, Zm, Ym
    Imperial: th, in, ft, yd, mi
    Other: nmi

Mass: 
    Metrix: mg, µg, ng, pg, kg, Mg, tonne
        dg, cg, fg, ag, zg, yg, dag, hg, Mg, Gg, Tg, Pg, Eg, Zg, Yg
    Imperial: oz, lb, ton
    Other: mol
    
Temparature: K, C, F, R

Luminosity: cd, lx

Force: N, kN, lbf, pdl

"""
None

# Classes

## Token Matching

### TokenOperandDefinition

In [4]:
class TokenOperandDefinition():
    """Defines an operand token that can be matched with using a string search"""
    
    def __init__(self, token, precedence):
        """Arguments:
            str   token:       string that identifies the operand
            float precedence:  precedence in order of operations (higher is executed first)
        """
        
        self.token = token
        self.precedence = float(precedence)
        
    def __repr__(self):
        return str(self)
        
    def __str__(self):
        return f'TokenOperandDefinition({self.token}, {self.precedence})'

### TokenGroupDefinition

In [5]:
class TokenGroupDefinition():
    """Defines an grouping token pair that can be matched with using a string search"""
    
    def __init__(self, open_token, close_token, seperators, ignore_missing, function):
        """Arguments:
            str      open_token:          string that identifies the opening token
            str      close_token:         string that identifies the closing token (can be the same as open_token)
            dict(str: depth) seperators:  allowed seperators and their seperating depth
            function function:            the function to convert the list of items into an in environment object
        """
        
        self.open_token = open_token
        self.close_token = close_token
        self.seperators = seperators
        self.ignore_missing = ignore_missing
        
        self.function = function
        
    def __repr__(self):
        return str(self)
        
    def __str__(self):
        return f'TokenGroupDefinition({self.open_token}, {self.close_token})'

### TokenNewItemDefinition

In [6]:
class TokenNewItemDefinition():
    """Defines an item seperation token that can be matched with using a string search"""
    
    def __init__(self, token):
        """Arguments:
            str token:  string that identifies the seperator
        """
        
        self.token = token
        
    def __repr__(self):
        return str(self)
        
    def __str__(self):
        return f'TokenNewItemDefinition({self.token}, {self.levels})'

## Token Tree Nodes

### Abstract NodeToken

In [7]:
class NodeToken():
    """represents a value, operation or grouping node that returns an evaluabe that can be evaluated to return a value"""
    
    def is_complete(self):
        """Returns whether this token is complete, if false, it is not a valud token and interpreting failed"""
        
        raise Exception('is_complete() not implemented')
        
    def set_left(self, node):
        """sets the left child node if it exists"""
        
        raise Exception('set_left(node) not implemented')

    def set_right(self, node):
        """sets the right child node if it exists"""
        
        raise Exception('set_right(node) not implemented')
        
    def get_right(self):
        """segetsts the right child node if it exists"""
        
        raise Exception('get_right() not implemented')
        
    def get_evaluable(self):
        """return an Evaluable obejct from this token, the token should be complete before this method is called"""
        
        raise Exception('get_evaluable() not implemented')

### NodeBinary

In [8]:
class NodeBinary(NodeToken):
    """represents a binary operation with a left and right child"""
    
    def __init__(self, operation_definition, parent=None):
        self.parent = parent
        self.operation_definition = operation_definition
        self.precedence = operation_definition.precedence
        self.left = None
        self.right = None
    
    def is_complete(self):
        return self.left != None and self.right != None
        
    def set_left(self, node):
        """sets left child"""
        
        self.left = node
        node.parent = self

    def set_right(self, node):
        """sets right child"""
        
        self.right = node
        node.parent = self
        
    def get_right(self):
        """returns right child"""
        
        return self.right
    
    def get_evaluable(self):
        """returns Evaluable that computes a result from both child tokens"""
        
        return VariableFunction(self.operation_definition.token, (self.left.get_evaluable(), self.right.get_evaluable()))
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeBinary({self.operation_definition}, left={self.left}, right={self.right})'

### NodeUnaryLeft

In [9]:
class NodeUnaryLeft(NodeToken):
    """represents a unary operator to the left of its operand with a single child"""
    
    def __init__(self, operation_definition, parent=None):
        self.parent = parent
        self.operation_definition = operation_definition
        self.precedence = operation_definition.precedence
        self.child = None
    
    def is_complete(self):
        return self.child != None
        
    def set_left(self, node):
        """sets single child"""
        
        self.child = node
        node.parent = self

    def set_right(self, node):
        """sets single child"""
        
        self.child = node
        node.parent = self
        
    def get_right(self):
        """gets single child"""
        
        return self.child
    
    def get_evaluable(self):
        """returns Evaluable that computes a result from the child token"""
        
        return VariableFunction(self.operation_definition.token, (self.child.get_evaluable(), ))
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeUnaryLeft({self.operation_definition}, child={self.child})'

### NodeUnaryRight

In [10]:
class NodeUnaryRight(NodeToken):
    """represents a unary operator to the right of its operand with a single child"""
    
    def __init__(self, operation_definition, parent=None):
        self.parent = parent
        self.operation_definition = operation_definition
        self.precedence = operation_definition.precedence
        self.child = None
    
    def is_complete(self):
        return self.child != None
        
    def set_left(self, node):
        """sets single child"""
        
        self.child = node
        node.parent = self

    def set_right(self, node):
        """sets single child"""
        
        self.child = node
        node.parent = self
        
    def get_right(self):
        """gets single child"""
        
        return self.child
    
    def get_evaluable(self):
        """returns Evaluable that computes a result from the child token"""
        
        return VariableFunction(self.operation_definition.token, (self.child.get_evaluable(), ))
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeUnaryRight({self.operation_definition}, child={self.child})'

### NodeGroup

In [11]:
class NodeGroup(NodeToken):
    """represents a grouping"""
    
    def __init__(self, group_definition, parent=None):
        self.parent = parent
        self.children = []
        
        self.group_definition = group_definition
        self.complete = False
        
        self.shape = {}
        self.sep_count = 0
    
    def is_complete(self):
        return self.complete

    def close(self):
        self.sep_count = 0
        self.complete = True
        
        expected_size = 1
        for d,s in self.shape.items(): expected_size *= s
        
        if not self.group_definition.ignore_missing:
            assert len(self.children) == expected_size, f'Inconsistent dimensions shape={self.shape}, expected={expected_size}, actual={len(self.children)}'
    
    def set_right(self, node):
        self.children[-1] = node
        node.parent = self
        
    def get_right(self):
        return self.children[-1]

    def add(self, node):
        self.children.append(node)
        node.parent = self
        
        if len(self.shape) > 0 and self.sep_count >= len(self.shape):
            for i in range(len(self.shape), self.sep_count+1):
                self.shape[i-1] = self.shape.get(i-1, 1)
            self.shape[self.sep_count-1] += 1
            
        self.sep_count = 0
        
    def increase(self, depth=1):
        self.sep_count += depth
        
        if len(self.shape) == 0:
            self.shape[0] = 1
        
        # ensure minumum number of dimensions exists
        for i in range(len(self.shape), self.sep_count+1):
            self.shape[i-1] = self.shape.get(i-1, 1)
    
    def get_shape(self):
        shape = list(self.shape.items())
        shape.sort(key=lambda x:x[0], reverse=True)
        shape = tuple(s for d,s in shape)
        return shape
    
    def get_children(self):
        return self.children
    
    def get_evaluable(self):
        
        evaluables = [v.get_evaluable() for v in self.children]
        
        return VariableTensor(evaluables, self.get_shape(), self.group_definition.function)
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeGroup({self.group_definition}, children={self.children})'

### NodeValue

In [12]:
class NodeValue(NodeToken):
    """represents a value"""
    
    def __init__(self, value, value_type, parent=None):
        self.parent = parent
        self.value = value
        self.value_type = value_type
    
    def is_complete(self):
        return True
    
    def get_evaluable(self):
        """returns the approriate Evaluable for the tyoe of value this node holds"""

        value = None
        if self.value_type == TOKEN_TYPE_STRING:
            trimmed_string = self.value.strip(self.value[0])
            value = String(trimmed_string)
            value = VariableTensor([value], ())
            
        if self.value_type == TOKEN_TYPE_INTEGER:
            value = Integer(self.value)
            value = VariableTensor([value], ())
            
        if self.value_type == TOKEN_TYPE_NUMBER:
            value = Real(self.value)
            value = VariableTensor([value], ())
            
        if self.value_type == TOKEN_TYPE_LITERAL:
            value = Variable(self.value)
        
        return value
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeValue({self.value}, {self.value_type})'

## Evaluable Tree Nodes

### Abstract Evaluable

In [13]:
class Evaluable():
    """represents a node that can be evaluated"""
    
    def eval(self, environment, **kwargs):
        """performs the final evaluation returning an in environment value
        
        Arguments:
            Environment environment:  holds the current environment variables
                           **kwargs:  any arguments the Evaluable requires to evaluate
        """
        
        raise Exception('eval(environment) not implemented')

### VariableFunction

In [14]:
class VariableFunction(Evaluable):
    
    def __init__(self, name, parameters):
        
        self.dtype = Evaluable
        self.name = name
        self.parameters = parameters
        if isinstance(parameters, VariableTensor):
            self.parameters = parameters.data
    
    def eval(self, environment, evaluable=False, **kwargs):
        
        if evaluable:
            return self
        
        assert self.name in environment, f'Function {self.name} not found'
        assert type(environment[self.name]) == FunctionSet, f'{self.name} is not a function'
        
        return environment[self.name].eval(environment, self.parameters)
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'VariableFunction_{self.name}({self.parameters})'

### Variable

In [15]:
class Variable(Evaluable):
    
    def __init__(self, name):
        
        self.dtype = Evaluable
        self.name = name
    
    def eval(self, environment, references=False, evaluable=False, **kwargs):
        
        if references:
            return Tensor(Reference(self.name), ())
        
        if evaluable:
            return self
        
        if self.name not in environment:
            raise Exception(f'Variable {self.name} not found')
        
        return environment[self.name]
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f"Variable({self.name})"

### VariableTensor

In [16]:
class VariableTensor(Evaluable):
    
    def __init__(self, data, shape, function=None):
        
        self.dtype = Evaluable
        self.data = data
        self.shape = shape
        self.function = function
    
    def eval(self, environment, evaluable=False, **kwargs):
        
        if evaluable:
            return self
        
        values = [v.eval(environment, **kwargs) for v in self.data]
        
        if self.function == None and self.shape == ():
            return Tensor(values[:1], self.shape)
        else:
            return self.function(environment, values, self.shape, **kwargs)
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'VariableTensor({self.data}, {self.shape})'

## Data Structures

### Tensor

In [17]:
def nd_index_from_shape(int_index, shape):
    devisor = 1
    devisors = [] if len(shape) == 0 else [devisor]
    for s in reversed(shape[1:]):
        devisor *= s
        devisors.append(devisor)
    index = []
    for devisor in reversed(devisors):
        i = int_index // devisor
        index.append(i)
        int_index -= i * devisor
    return index

In [18]:
def count_end_zeros(data, sub=None):
    if sub:
        data = [d-s+1 for [d,s] in zip(data, sub)]
    c = 0
    for i in reversed(data):
        if i == 0:
            c += 1
        else:
            break
    return c

In [19]:
class Tensor(Evaluable):
    """
    represents a value/tensor of any type, shape and rank
    """
    def __init__(self, data, shape=()):
        
        data = np.array(data, object)
        data = data.reshape((data.size,))
        
        assert data.size > 0, 'Tensor cannot be empty'
        
        if all(type(t) == Tensor for t in data):
            inner_shape = data[0].shape
            assert all(t.shape == inner_shape for t in data), 'Cannot combine tensors of different shapes'
            
            data = np.concatenate([t.data for t in data])
            shape = shape + inner_shape
        
        self._data = np.array(data, object).reshape(shape)
        self.data = self._data.reshape((self._data.size))
        
        self.first = self.data[0]
        self.shape = self._data.shape
        self.size = self._data.size
        self.rank = self._data.ndim
        self.dtype = type(self.data[0])
        
        assert all([type(i) == self.dtype for i in self.data]), 'All items in a Tensor must be of the same type'
        
    def __getitem__(self, index):
        
        data = self.data[index] if type(index) == int else self._data[index]
        shape = data.shape if type(data) == np.ndarray else ()
            
        return Tensor(data, shape)
        
    def __setitem__(self, index, value):
        
        if type(value) == Tensor:
            value = value._data
        
        if type(index) == int:
            self.data[index] = value
        else:
            self._data[index] = value
    
    def eval(self, environment, **kwargs):
        return self
    
    def __repr__(self):
        if self.rank == 0:
            return f'Tensor({self.data[()]})'
        else:
            return f'Tensor({self.data}, {self.shape})'
    
    def __str__(self):
        if self.rank == 0:
            return str(self.first)
        else:
            shape = list(self.shape)
            inner_size = shape.pop()
            strings = [str(i) for i in self.data]
            inner_lists = list(zip(*[iter(strings)]*inner_size))
            strings = ['['+', '.join(l)+']' for l in inner_lists]
            
            indexes = [nd_index_from_shape(i, shape) for i in range(len(strings))]
            
            left = [count_end_zeros(d) for d in indexes]
            left = [(len(shape) - i) * ' ' + i * '[' for i in left]

            right = [count_end_zeros(d, shape) for d in indexes]
            right = [i * ']' + ',' + i * '\n' for i in right]

            strings = [l+s+r for (l,s,r) in zip(left, strings, right)]
            strings[-1] = strings[-1].strip('\n,')

            string = '\n'.join(strings)
        
            return string

### Array

In [20]:
class Array(Evaluable):
    
    def __init__(self, data):
        self.data = np.array(data, object)
        self.data = self.data.reshape((self.data.size,))
        self.size = self.data.size
        self.dtype = Array
            
    def __getitem__(self, index):
        if type(index) == int:
            return self.data[index]
        elif type(index) in [tuple, list]:
            if len(index) == 0:
                raise Exception('No index given in array item lookup')
            elif len(index) == 1:
                return self.data[index[0]]
            else:
                raise Exception('Too many indices given in array item lookup')
        else:
            return Array(self.data[index])
        
    def __setitem__(self, index, value):
        self.data[index] = value
    
    def eval(self, environment, **kwargs):
        return self
    
    def __repr__(self):
        return f'Array({self.data[()]})'
    
    def __str__(self):
        items = [str(i) for i in self.data]
        items = ', '.join(items)
        return f'({items})'

### FunctionSet

In [21]:
class FunctionSet(Evaluable):
    
    def __init__(self, name, evaluation_paremeters=None):
        self.name = name
        self.signatures = {}
        self.evaluation_paremeters = evaluation_paremeters or {}
        
        self.dtype = FunctionSet
        
    def __contains__(self, partial_signature):
        l = len(partial_signature)
        for signature in self.signatures:
            if signature[:l] == partial_signature:
                return True
            
        return False
    
    def __getitem__(self, signature):
        return self.signatures[signature]
    
    def __setitem__(self, signature, value):
        par_names, function = value
        self.signatures[signature] = (par_names, function)
        
    def add(self, function_signature):
        self.signatures[function_signature.parameter_types] = function_signature
        
    def eval(self, environment, parameters, **kwargs):
        
        parameters = list(parameters)
        signature = [type(p) for p in parameters]
        
        for i, parameter in enumerate(parameters):
            eval_params = self.evaluation_paremeters.get(i, {})
            parameters[i] = parameter.eval(environment, **eval_params)
            signature[i] = parameters[i].dtype
        
        signature = tuple(signature)
        function_signature = None
        
        for signature_candidate in self.signatures:
            if len(signature) == len(signature_candidate) and all([a==b or a==None for a,b in zip(signature_candidate, signature)]):
                function_signature = self.signatures[signature_candidate]
        
#         assert signature in self.signatures, f'Signature {self.name}{tuple(p.__name__ for p in signature)} has no matching overload'
        assert function_signature != None, f'Signature {self.name}{tuple(p.__name__ for p in signature)} has no matching overload'
            
#         function_signature = self.signatures[signature]
        
        function = function_signature.function
        parameter_names = function_signature.parameter_names
        
        actual_shapes = [p.shape if type(p) == Tensor else () for p in parameters]
        actual_ranks = [len(s) for s in actual_shapes]
        
        ranks = [r if r!=None else a for r,a in zip(function_signature.parameter_ranks, actual_ranks)]
        extended_shapes = [s if r==0 else s[:-r] for s,r in zip(actual_shapes, ranks)]
        
        extended_shape = reduce(lambda x,y: x if len(x) > len(y) else y, extended_shapes)
        extended_ranks = [len(s) for s in extended_shapes]
        
        assert all([s in [extended_shape, ()] for s in extended_shapes]), f'Ranks, shape or extended shapes do not match'
        
        if extended_shape == ():
            return self.runFunction(function, environment, parameter_names, parameters)
        
        else:
            data = np.empty(extended_shape, dtype=object)
            
            for i in product(*[range(d) for d in extended_shape]):
                extracted_parameters = [p if r == 0 else p[i] for p, r in zip(parameters, extended_ranks)]
                
                data[i] = self.runFunction(function, environment, parameter_names, extracted_parameters)._data.tolist()
                    
            data = np.array(data.tolist())
            
            return Tensor(data, data.shape)
        
    def runFunction(self, function, environment, parameter_names, parameters):
        
        if parameter_names == False or parameter_names == None:
            return function(*parameters)
        
        elif parameter_names == True:
            return function(environment, *parameters)

        else:
            new_scoped_environment = environment.enterScope({k:v for k,v in zip(parameter_names, parameters)})
            return function(new_scoped_environment)
        
    def __repr__(self):
        return f'FunctionSet({self.name}, {self.signatures})'
    
    def __str__(self):
        if len(self.signatures) == 0:
            parameter_types = ''
        if len(self.signatures) == 1:
            parameter_types = next(iter(self.signatures))
            parameter_types = ['Any' if t == None else t.__name__ for t in parameter_types]
            parameter_types = ', '.join(parameter_types)
        else:
            parameter_types = '...'
            
        name = '<AnonymousFunction>' if self.name == '' else self.name
            
        return f'{name}({parameter_types})'

### ConversionFunctionSet

In [22]:
class ConversionFunctionSet(Evaluable):
    
    def __init__(self, name, forwards, backwards):
        self.name = name
        self.forwards = forwards
        self.backwards = backwards
        self.dtype = ConversionFunctionSet
    
    def eval(self, environment, tensor, forwards=True, **kwargs):
        shape = tensor.shape
        data = tensor.data
        
        if forwards:
            converted_data = [self.forwards(d) for d in data]
        else:
            converted_data = [self.backwards(d) for d in data]
            
        return Tensor(converted_data, shape)
    
    def __repr__(self):
        return f'ConversionFunctionSet({self.name})'
    
    def __str__(self):
        return f'<{self.name}>'

## Data Types

### Abstract Data

In [23]:
class Data(Evaluable):
    pass

### String

In [24]:
class String(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment, **kwargs):
        return self
    
    def __repr__(self):
        return f'String("{self.value}")'
    
    def __str__(self):
        return f'"{self.value}"'

### Integer

In [25]:
class Integer(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment, **kwargs):
        return self

### Real

In [26]:
class Real(Data):
    
    def __init__(self, value):
        self.value = Decimal(value)
    
    def eval(self, environment, **kwargs):
        return self
    
    def __repr__(self):
        return f'Real({str(self.value)})'
    
    def __str__(self):
        return str(self.value)

### Complex

### Boolean

In [27]:
class Boolean(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment, **kwargs):
        return self

### FunctionSignature

In [28]:
class FunctionSignature():
    
    def __init__(self, name, function, parameter_names, parameter_types, parameter_ranks=None, parameter_shapes=None):
        
        self.name = name
        self.function = function
        self.parameter_count = len(parameter_types)
        self.parameter_names = parameter_names
        self.parameter_types = parameter_types
        self.parameter_ranks = parameter_ranks or self.parameter_count*(None,)
        self.parameter_shapes = parameter_shapes or self.parameter_count*(None,)
        
    def __repr__(self):
        return f'FunctionSignature{(self.name, self.function, self.parameter_names, self.parameter_types, self.parameter_ranks, self.parameter_shapes)}'
    
    def __str__(self):
        parameter_types = [str(t) for t in self.parameter_types]
        parameter_types = ', '.join(parameter_types)
        return f'{self.name}({parameter_types})'

### Reference

In [29]:
class Reference(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment, **kwargs):
        return self

# Built In Functions, Operators and Variables

### BuiltIns

In [30]:
class BuiltIns():
    
    def __init__(self):
        self.functions = []
        self.function_parameter_evaluation_parameters = {}
        self.vars = []
        self.binary_operators = []
        self.left_unary_operators = []
        self.right_unary_operators = []
        self.groups = []
        self.new_items = []
        self.conversion_function_sets = []
        
    def register_function(self, name, function, parameter_names, parameter_types, parameter_ranks=None, parameter_shapes=None):
        self.functions.append((name, function, parameter_names, parameter_types, parameter_ranks, parameter_shapes))
        
    def set_evaluation_parameter(self, function_name, parameter_index, key, value):
        parameter_list = self.function_parameter_evaluation_parameters.get(function_name, {})
        parameters = parameter_list.get(parameter_index, {})
        parameters[key] = value
        parameter_list[parameter_index] = parameters
        self.function_parameter_evaluation_parameters[function_name] = parameter_list
        
    def register_binary_operator(self, name, precedence):
        self.binary_operators.append(TokenOperandDefinition(name, precedence))
        
    def register_left_unary_operator(self, name, precedence):
        self.left_unary_operators.append(TokenOperandDefinition(name, precedence))
        
    def register_right_unary_operator(self, name, precedence):
        self.right_unary_operators.append(TokenOperandDefinition(name, precedence))
        
    def register_var(self, name, value):
        self.vars.append((name, value))
        
    def register_grouping(self, open_token, close_token, seperators, ignore_missing, function):
        self.groups.append(TokenGroupDefinition(open_token, close_token, seperators, ignore_missing, function))
        
    def register_new_item_seperator(self, token):
        self.new_items.append(TokenNewItemDefinition(token))
        
    def register_conversion_function_set(self, name, forwards, backwards):
        self.conversion_function_sets.append((name, forwards, backwards))
        
        
built_ins = BuiltIns()

### Groups

#### Group ( )

In [31]:
def group_round(environment, items, shape, force_array=False, **kwargs):
    
    if shape == () and force_array == False:
        return items[0]
    else:
        return Array(items)

built_ins.register_grouping('(', ')', {',': 1, '\n': 1}, True, group_round)

#### Group [ ]

In [32]:
def group_squre(environment, items, shape, **kwargs):
    
    assert len(items) > 0, 'Tensors can not be empty'
    
    base_shape = items[0].shape
    assert all([i.shape == base_shape for i in items]), 'Tensor elements have inconsistent shapes'
    
    extended_shape = shape + base_shape
    data = np.array([i.data for i in items]).reshape(extended_shape)
    
    return Tensor(data, extended_shape)

built_ins.register_grouping('[', ']', {',': 1, ';': 2, '\n': 0}, False, group_squre)

#### Group { }

In [33]:
def group_curly(environment, items, shape, **kwargs):
    return items[-1]

built_ins.register_grouping('{', '}', {';': 1, '\n': 1}, True, group_curly)

#### Group | |

In [34]:
def group_straight(environment, items, shape, **kwargs):
    
    assert shape == (), 'Cannot take absolute value of an array'
    assert items[0].dtype == Real, 'Cannot take absolute value of non Real value'
    
#     return Tensor(Real(abs(items[0].first.value)))

    return VariableFunction('abs', items).eval(environment)

built_ins.register_grouping('|', '|', {}, True, group_straight)
built_ins.register_grouping('||', '||', {}, True, group_straight)

### New Item Seperators

In [35]:
built_ins.register_new_item_seperator(',')
built_ins.register_new_item_seperator(';')
built_ins.register_new_item_seperator('\n')

### Variabels

In [36]:
built_ins.register_var('PI', Tensor([Real('3.141592653589793238462643383279502884197')]))
built_ins.register_var('e', Tensor([Real('2.718281828459045235360287471352662497757')]))
built_ins.register_var('phi', Tensor([Real('1.61803398874989484820458683436563811772')]))

### Declare Operations

In [37]:
# Unary Operations: -, +
built_ins.register_left_unary_operator('+', 7)
built_ins.register_left_unary_operator('-', 7)

# Binary Operations:

# arithmetic
built_ins.register_binary_operator('+',   3)
built_ins.register_binary_operator('-',   3)
built_ins.register_binary_operator('*',   4)
built_ins.register_binary_operator('4',   4)
built_ins.register_binary_operator('/',   4)
built_ins.register_binary_operator('//',  4)
built_ins.register_binary_operator('%',   4)
built_ins.register_binary_operator('mod', 4)
built_ins.register_binary_operator('^',   5)

# matrix
built_ins.register_binary_operator('#',   4)
built_ins.register_binary_operator('matmul', 4)

# vector
built_ins.register_binary_operator('.*' , 4)
built_ins.register_binary_operator('dot', 4)

# root
built_ins.register_left_unary_operator('$', 6)
built_ins.register_binary_operator('$', 6)

# function call
built_ins.register_binary_operator('9', 9)
built_ins.set_evaluation_parameter('9', 1, 'force_array', True)

# function definition
built_ins.register_binary_operator('=>', 1)
built_ins.set_evaluation_parameter('=>', 0, 'references', True)
built_ins.set_evaluation_parameter('=>', 1, 'evaluable', True)

built_ins.register_binary_operator('=', 0)
built_ins.set_evaluation_parameter('=', 0, 'references', True)

# conversion
built_ins.register_binary_operator('@', -1)

### Register Functions

#### Assignment: =

In [38]:
def assignment(environment, key, value):
    
    if type(key) == Array:
        for k, v in zip(key, value):
            assignment(environment, k, v)
    else:
        environment[key.first.value] = value
        
    return value

built_ins.register_function('=', assignment, True, (Array, None))
built_ins.register_function('=', assignment, True, (Reference, None))

#### Function Definition: =>

In [39]:
def define_function(environment, parameters, evaluable):
    
    if type(parameters) == Tensor and parameters.dtype == Reference:
        parameter_names = [parameters.first.value]
    elif type(parameters) == Array:
        parameter_names = [p.first.value for p in parameters.data]
    
    parameter_types = tuple([None] * len(parameter_names))
    
    def function(environment):
        return evaluable.eval(environment)
    
    function_signature = FunctionSignature('', function, parameter_names, parameter_types, parameter_ranks=None, parameter_shapes=None)

    function_set = FunctionSet('')
    function_set.add(function_signature)
    
    return function_set

built_ins.register_function('=>', define_function, True, (Reference, None))
built_ins.register_function('=>', define_function, True, (Array, None))

#### Unary Operations: -, +

In [40]:
built_ins.register_function('+', lambda t:Tensor(Real(+t.first.value)), None, (Real,), (0,))
built_ins.register_function('-', lambda t:Tensor(Real(-t.first.value)), None, (Real,), (0,))

#### Binary Operantors: +, -, *, /, //, %, ^

In [41]:
# A + B
built_ins.register_function('+', lambda a, b: Tensor(Real(a.first.value + b.first.value)), None, (Real, Real), (0, 0))

# A - B
built_ins.register_function('-', lambda a, b: Tensor(Real(a.first.value - b.first.value)), None, (Real, Real), (0, 0))

# A * B
built_ins.register_function('*', lambda a, b: Tensor(Real(a.first.value * b.first.value)), None, (Real, Real), (0, 0))
# built_ins.register_function('9', lambda a, b: Tensor(Real(a.first.value * b.first.value)), None, (Real, Real), (0, 0))

# A / B
built_ins.register_function('/', lambda a, b: Tensor(Real(a.first.value / b.first.value)), None, (Real, Real), (0, 0))

# A // B
built_ins.register_function('//', lambda a, b: Tensor(Real(a.first.value // b.first.value)), None, (Real, Real), (0, 0))

# A % B
built_ins.register_function('%', lambda a, b: Tensor(Real(a.first.value % b.first.value)), None, (Real, Real), (0, 0))
built_ins.register_function('mod', lambda a, b: Tensor(Real(a.first.value % b.first.value)), None, (Real, Real), (0, 0))

# A ^ B
built_ins.register_function('^', lambda a, b: Tensor(Real(a.first.value ** b.first.value)), None, (Real, Real), (0, 0))


#### Matrix Multiplication: # #

In [42]:
def matmul(A, B):
    
    a_shape = A.shape
    b_shape = B.shape
    
    assert A.rank >= 2 and B.rank >= 2, 'left and right sides must be matrices'
    assert a_shape[-1] == b_shape[-2], 'left n_cols must equal right n_rows'
    
    n_rows = a_shape[0]
    n_cols = b_shape[1]
    n_vec = a_shape[1]
    shape = (n_rows , n_cols)
    
    data = np.empty(shape, dtype=object)
    
    for row in range(n_rows):
        for col in range(n_cols):
            v = 0
            for i in range(n_vec):
                v += A[(row, i)].first.value * B[(i, col)].first.value
            data[row,col] = Real(v)
    
    return Tensor(data, shape)

built_ins.register_function('#', matmul, None, (Real, Real), (2, 2))
built_ins.register_function('matmul', matmul, None, (Real, Real), (2, 2))

#### Vector Dot Product: .*

In [43]:
def dot(A, B):
    
    a_shape = A.shape
    b_shape = B.shape
    
    assert A.rank == 1 and B.rank == 1, 'left and right sides must be vectors'
    assert a_shape == b_shape, 'vector length must match'
    
    n_vec = a_shape[-1]
    
    v = 0
    for i in range(n_vec):
        v += A[(i,)].first.value * B[(i,)].first.value
    
    return Tensor(Real(v))

built_ins.register_function('.*', dot, None, (Real, Real), (1, 1))
built_ins.register_function('dot', dot, None, (Real, Real), (1, 1))

#### Absolute value abs

In [44]:
def absolute(A):
    return Tensor(Real(abs(A.first.value)))

built_ins.register_function('abs', absolute, None, (Real,), (0,))

#### Root and Square root $

In [45]:
def root(A, B):
    return Tensor(Real(B.first.value ** (Decimal(1) / A.first.value)))

def sqrt(A):
    return Tensor(Real(A.first.value ** Decimal(0.5)))

built_ins.register_function('$', sqrt, None, (Real,), (0,))
built_ins.register_function('$', root, None, (Real, Real), (0, 0))

#### Function Call

In [46]:
def function_call(environment, function_set, parameters):
    return function_set.eval(environment, parameters)

built_ins.register_function('9', function_call, True, (FunctionSet, Array))


def foo(A):
    return Tensor(Real(A.first.value * A.first.value))
def bar(A, B):
    return Tensor(Real(A.first.value * B.first.value))
    
built_ins.register_function('fun', foo, False, (Real,), (0,))
built_ins.register_function('fun', bar, False, (Real, Real), (0, 0))

#### Tensor Lookup

In [47]:
def tensor_lookup(environment, tensor, parameters):
    index = [int(p.first.value) for p in parameters]
    return tensor[tuple(index)]

built_ins.register_function('9', tensor_lookup, True, (Real, Array))

#### Array Lookup

In [48]:
def array_lookup(environment, array, parameters):
    index = [int(p.first.value) for p in parameters]
    return array[tuple(index)]

built_ins.register_function('9', array_lookup, True, (Array, Array))

#### Conversion @, [implocit]

In [49]:
def conversion_forwards_function_call(environment, tensor, conversion_function_set):
    return conversion_function_set.eval(environment, tensor, True)

def conversion_backwards_function_call(environment, tensor, conversion_function_set):
    return conversion_function_set.eval(environment, tensor, False)

built_ins.register_function('9', conversion_forwards_function_call, True, (None, ConversionFunctionSet))
built_ins.register_function('@', conversion_backwards_function_call, True, (None, ConversionFunctionSet))

### Conversion Specifiers

#### Distance

In [50]:
def km_forwards(km):
    return Real(km.value * Decimal(1000))

def km_backwards(m):
    return Real(m.value / Decimal(1000))

def  m_forwards(m):
    return Real(m.value)

def  m_backwards(m):
    return Real(m.value)

def cm_forwards(cm):
    return Real(cm.value / Decimal(100))

def cm_backwards(m):
    return Real(m.value * Decimal(100))

def mm_forwards(mm):
    return Real(mm.value / Decimal(1000))

def mm_backwards(m):
    return Real(m.value * Decimal(1000))

def um_forwards(um):
    return Real(um.value / Decimal(1000000))

def um_backwards(m):
    return Real(m.value * Decimal(1000000))

def nm_forwards(nm):
    return Real(nm.value / Decimal(1000000000))

def nm_backwards(m):
    return Real(m.value * Decimal(1000000000))

def pm_forwards(pm):
    return Real(pm.value / Decimal(1000000000000))

def pm_backwards(m):
    return Real(m.value * Decimal(1000000000000))

built_ins.register_conversion_function_set('km', km_forwards, km_backwards)
built_ins.register_conversion_function_set( 'm',  m_forwards,  m_backwards)
built_ins.register_conversion_function_set('cm', cm_forwards, cm_backwards)
built_ins.register_conversion_function_set('mm', mm_forwards, mm_backwards)
built_ins.register_conversion_function_set('um', um_forwards, um_backwards)
built_ins.register_conversion_function_set('nm', nm_forwards, nm_backwards)
built_ins.register_conversion_function_set('pm', pm_forwards, pm_backwards)

#### Area

In [51]:
def km2_forwards(km):
    return Real(km.value * Decimal(1000**2))

def km2_backwards(m):
    return Real(m.value / Decimal(1000**2))

def  m2_forwards(m):
    return Real(m.value)

def  m2_backwards(m):
    return Real(m.value)

def cm2_forwards(cm):
    return Real(cm.value / Decimal(100**2))

def cm2_backwards(m):
    return Real(m.value * Decimal(100**2))

def mm2_forwards(mm):
    return Real(mm.value / Decimal(1000**2))

def mm2_backwards(m):
    return Real(m.value * Decimal(1000**2))

def um2_forwards(um):
    return Real(um.value / Decimal(1000000**2))

def um2_backwards(m):
    return Real(m.value * Decimal(1000000**2))

def nm2_forwards(nm):
    return Real(nm.value / Decimal(1000000000**2))

def nm2_backwards(m):
    return Real(m.value * Decimal(1000000000**2))

def pm2_forwards(pm):
    return Real(pm.value / Decimal(1000000000000**2))

def pm2_backwards(m):
    return Real(m.value * Decimal(1000000000000**2))

built_ins.register_conversion_function_set('km2', km2_forwards, km2_backwards)
built_ins.register_conversion_function_set( 'm2',  m2_forwards,  m2_backwards)
built_ins.register_conversion_function_set('cm2', cm2_forwards, cm2_backwards)
built_ins.register_conversion_function_set('mm2', mm2_forwards, mm2_backwards)
built_ins.register_conversion_function_set('um2', um2_forwards, um2_backwards)
built_ins.register_conversion_function_set('nm2', nm2_forwards, nm2_backwards)
built_ins.register_conversion_function_set('pm2', pm2_forwards, pm2_backwards)

#### Volume

In [52]:
def km3_forwards(km):
    return Real(km.value * Decimal(1000**3))

def km3_backwards(m):
    return Real(m.value / Decimal(1000**3))

def  m3_forwards(m):
    return Real(m.value)

def  m3_backwards(m):
    return Real(m.value)

def   l_forwards(l):
    return Real(l.value / Decimal(10**3))

def   l_backwards(m):
    return Real(m.value * Decimal(10**3))

def cm3_forwards(cm):
    return Real(cm.value / Decimal(100**3))

def cm3_backwards(m):
    return Real(m.value * Decimal(100**3))

def mm3_forwards(mm):
    return Real(mm.value / Decimal(1000**3))

def mm3_backwards(m):
    return Real(m.value * Decimal(1000**3))

def um3_forwards(um):
    return Real(um.value / Decimal(1000000**3))

def um3_backwards(m):
    return Real(m.value * Decimal(1000000**3))

def nm3_forwards(nm):
    return Real(nm.value / Decimal(1000000000**3))

def nm3_backwards(m):
    return Real(m.value * Decimal(1000000000**3))

def pm3_forwards(pm):
    return Real(pm.value / Decimal(1000000000000**3))

def pm3_backwards(m):
    return Real(m.value * Decimal(1000000000000**3))

built_ins.register_conversion_function_set('km3', km3_forwards, km3_backwards)
built_ins.register_conversion_function_set( 'm3',  m3_forwards,  m3_backwards)
built_ins.register_conversion_function_set(  'l',   l_forwards,   l_backwards)
built_ins.register_conversion_function_set('cm3', cm3_forwards, cm3_backwards)
built_ins.register_conversion_function_set( 'ml', cm3_forwards, cm3_backwards)
built_ins.register_conversion_function_set('mm3', mm3_forwards, mm3_backwards)
built_ins.register_conversion_function_set('um3', um3_forwards, um3_backwards)
built_ins.register_conversion_function_set('nm3', nm3_forwards, nm3_backwards)
built_ins.register_conversion_function_set('pm3', pm3_forwards, pm3_backwards)

#### Time

In [53]:
def  ms_forwards(ms):
    return Real(ms.value / Decimal(1000))

def  ms_backwards(s):
    return Real(s.value * Decimal(1000))

def   s_forwards(s):
    return Real(s.value)

def   s_backwards(s):
    return Real(s.value)

def min_forwards(min):
    return Real(min.value * Decimal(60))

def min_backwards(s):
    return Real(s.value / Decimal(60))

def  hr_forwards(hr):
    return Real(hr.value * Decimal(3600))

def  hr_backwards(s):
    return Real(s.value / Decimal(3600))

def day_forwards(day):
    return Real(day.value * Decimal(86400))

def day_backwards(s):
    return Real(s.value / Decimal(86400))

def time_forwards(string):
    pattern = r'(?:(\d+)(?:d|D)\s*)?(\d{1,2})(?:h|H)?:(\d{1,2})(?:m|M)?(?::(\d{1,2}(?:\.\d+)?)(?:s|S)?)?'
    match = re.fullmatch(pattern, string.value.strip())
    if match:
        d = match.group(1) or 0
        h = match.group(2) or 0
        m = match.group(3) or 0
        s = match.group(4) or 0
        return Tensor(Real( Decimal(d) * 86400 + Decimal(h) * 3600 + Decimal(m) * 60 + Decimal(s)))
    else:
        raise Exception(f'Invalid time formatting: "{string.value}"')

def time_backwards(t):
    t = t.value
    d = t // Decimal(86400)
    t -= d * Decimal(86400)
    h = t // Decimal(3600)
    t -= h * Decimal(3600)
    m = t // Decimal(60)
    t -= m * Decimal(60)
    s = t // Decimal(1)
    t -= s * Decimal(1)
    ms = t // Decimal('0.001')
    t -= ms * Decimal('0.001')
    ps = t / Decimal('0.000001')
    
    str_d = str(int(d))
    str_h = str(int(h)).zfill(2)
    str_m = str(int(m)).zfill(2)
    str_s = str(int(s)).zfill(2)
    str_ms = str(int(ms)).zfill(3)
    str_ps = str(int(ps)).zfill(3)
    
    str_days = '' if d == 0 else                    f'{str_d}d ' 
    str_time =                                      f'{str_h}h:{str_m}m:{str_s}'
    str_fraction_1 = '' if ms == 0 and ps == 0 else f'.{str_ms}'
    str_fraction_2 = '' if ps == 0 else             f'{str_ps}'
    
    return Tensor(String( str_days+str_time+str_fraction_1+str_fraction_2+'s' ))
        

built_ins.register_conversion_function_set(  'ms',   ms_forwards,   ms_backwards)
built_ins.register_conversion_function_set(   's',    s_forwards,    s_backwards)
built_ins.register_conversion_function_set( 'min',  min_forwards,  min_backwards)
built_ins.register_conversion_function_set(  'hr',   hr_forwards,   hr_backwards)
built_ins.register_conversion_function_set( 'day',  day_forwards,  day_backwards)
built_ins.register_conversion_function_set('time', time_forwards, time_backwards)

# Define Token

## Tokens Types

In [54]:
TOKEN_TYPE_STRING =   'string'
TOKEN_TYPE_INTEGER =  'integer'
TOKEN_TYPE_NUMBER =   'number'
TOKEN_TYPE_OPERATOR = 'operand'
TOKEN_TYPE_OPEN_GROUP = 'group'
TOKEN_TYPE_LITERAL =  'literal'

TOKEN_TYPE_OPEN_GROUP = 'open group'
TOKEN_TYPE_CLOSE_GROUP = 'close group'
TOKEN_TYPE_NEW_ITEM    = 'new item'
TOKEN_TYPE_BINARY      = 'binary operand'
TOKEN_TYPE_UNARY_LEFT  = 'left unary operand'
TOKEN_TYPE_UNARY_RIGHT = 'right unary operand'

OPERAND_TYPES = [
    TOKEN_TYPE_OPEN_GROUP,
    TOKEN_TYPE_CLOSE_GROUP, 
    TOKEN_TYPE_NEW_ITEM, 
    TOKEN_TYPE_BINARY, 
    TOKEN_TYPE_UNARY_LEFT, 
    TOKEN_TYPE_UNARY_RIGHT, 
]

TOKEN_TYPE_VALUE = [
    TOKEN_TYPE_STRING,
    TOKEN_TYPE_INTEGER,
    TOKEN_TYPE_NUMBER,
    TOKEN_TYPE_LITERAL,
]

## Token Operations

In [55]:
token_definitions = {
    TOKEN_TYPE_OPEN_GROUP:  built_ins.groups,
    TOKEN_TYPE_CLOSE_GROUP: built_ins.groups, 
    TOKEN_TYPE_NEW_ITEM:    built_ins.new_items, 
    TOKEN_TYPE_BINARY:      built_ins.binary_operators, 
    TOKEN_TYPE_UNARY_LEFT:  built_ins.left_unary_operators, 
    TOKEN_TYPE_UNARY_RIGHT: built_ins.right_unary_operators, 
}

## Operation Indexing

In [56]:
def find_token_definition(string, token_type, f = lambda x:x.token):
    for token_definition in token_definitions[token_type]:
        if string == f(token_definition):
#         if re.fullmatch(operator.re_match, string) != None:
            return token_definition
    return None

## Define Token Regular Expressions

In [57]:
def re_join(l, f=lambda x:x):
    items = [f(i) for i in l]
    items.sort(key=lambda x:len(x), reverse=True)
    return '(' + ')|('.join([re.escape(i) for i in items]) + ')'

# regex
re_number =    r"""((([\.][0-9]+)|([0-9]+[\.]?[0-9]*))([eE][-+]?[0-9]+)?)"""
re_integer =   r"""((0b|0o|0d|0x|[0-9]+_)[0-9a-zA-Z,]+)"""
re_string =    r"""((\""".*?\""")|('''.*?''')|(".*?")|('.*?'))"""
re_literal =   r"""([A-Za-z_][A-Za-z0-9_]*)"""


re_open_group =           re_join(built_ins.groups, lambda x:x.open_token)
re_close_group =          re_join(built_ins.groups, lambda x:x.close_token)
re_binary_operands =      re_join(built_ins.binary_operators, lambda x:x.token)
re_left_unary_operands =  re_join(built_ins.left_unary_operators, lambda x:x.token)
re_right_unary_operands = re_join(built_ins.right_unary_operators, lambda x:x.token)
re_new_item =             re_join(built_ins.new_items, lambda x:x.token)


re_tokens = {
    TOKEN_TYPE_STRING:      re_string,
    TOKEN_TYPE_INTEGER:     re_integer,
    TOKEN_TYPE_NUMBER:      re_number,
    TOKEN_TYPE_LITERAL:     re_literal,
    TOKEN_TYPE_OPEN_GROUP:  re_open_group,
    TOKEN_TYPE_BINARY:      re_binary_operands,
    TOKEN_TYPE_UNARY_LEFT:  re_left_unary_operands,
    TOKEN_TYPE_UNARY_RIGHT: re_right_unary_operands,
    TOKEN_TYPE_NEW_ITEM:    re_new_item,
    TOKEN_TYPE_CLOSE_GROUP: re_close_group,
}

re_tokens

{'string': '((\\""".*?\\""")|(\'\'\'.*?\'\'\')|(".*?")|(\'.*?\'))',
 'integer': '((0b|0o|0d|0x|[0-9]+_)[0-9a-zA-Z,]+)',
 'number': '((([\\.][0-9]+)|([0-9]+[\\.]?[0-9]*))([eE][-+]?[0-9]+)?)',
 'literal': '([A-Za-z_][A-Za-z0-9_]*)',
 'open group': '(\\|\\|)|(\\()|(\\[)|(\\{)|(\\|)',
 'binary operand': '(matmul)|(mod)|(dot)|(\\/\\/)|(\\.\\*)|(\\=\\>)|(\\+)|(\\-)|(\\*)|(4)|(\\/)|(\\%)|(\\^)|(\\#)|(\\$)|(9)|(\\=)|(\\@)',
 'left unary operand': '(\\+)|(\\-)|(\\$)',
 'right unary operand': '()',
 'new item': '(\\,)|(\\;)|(\\\n)',
 'close group': '(\\|\\|)|(\\))|(\\])|(\\})|(\\|)'}

# Parse

## Lexing

In [ ]:
def re_match_length(string, re_pattern):
    match = re.match(re_pattern, string)
    return match.span()[1] if match != None else 0

In [58]:
# TOKEN_TYPE_STRING
# TOKEN_TYPE_INTEGER
# TOKEN_TYPE_NUMBER
# TOKEN_TYPE_LITERAL

# TOKEN_TYPE_OPEN_GROUP
# TOKEN_TYPE_BINARY
# TOKEN_TYPE_UNARY_LEFT
# TOKEN_TYPE_UNARY_RIGHT
# TOKEN_TYPE_NEW_ITEM
# TOKEN_TYPE_CLOSE_GROUP

def lexing(string, ans_available=False):
    tokens = []

    i = 0
    while i < len(string):
        
        if string[i] in ' ':
            i += 1
            continue
            
        last_token_type = tokens[-1][0] if len(tokens) > 0 else None
        allowed_token_types = []
        allowed_token_types_with_implicit_op = {}
        allowed_token_types_with_ans = {}

        # begin with
        if last_token_type in [
            None,
        ]:
            
            allowed_token_types = [TOKEN_TYPE_BINARY] if ans_available else []
            allowed_token_types += [
                TOKEN_TYPE_UNARY_LEFT,
                TOKEN_TYPE_OPEN_GROUP,
                TOKEN_TYPE_NEW_ITEM,
                TOKEN_TYPE_CLOSE_GROUP,
                *TOKEN_TYPE_VALUE,
            ]
            if ans_available:
                allowed_token_types_with_ans = {
                    TOKEN_TYPE_UNARY_RIGHT: '',
                    TOKEN_TYPE_BINARY: '',
                }
        
        if last_token_type in [\
            TOKEN_TYPE_OPEN_GROUP,
            TOKEN_TYPE_NEW_ITEM,
        ]:
            allowed_token_types = [
                TOKEN_TYPE_UNARY_LEFT,
                TOKEN_TYPE_OPEN_GROUP,
                TOKEN_TYPE_NEW_ITEM,
                TOKEN_TYPE_CLOSE_GROUP,
                *TOKEN_TYPE_VALUE,
            ]

        # after value excluding literal
        if last_token_type in [
            TOKEN_TYPE_CLOSE_GROUP,
            TOKEN_TYPE_UNARY_RIGHT,
            TOKEN_TYPE_STRING,
            TOKEN_TYPE_INTEGER,
            TOKEN_TYPE_NUMBER,
        ]:
            allowed_token_types = [
                TOKEN_TYPE_UNARY_RIGHT,
                TOKEN_TYPE_BINARY,
                TOKEN_TYPE_NEW_ITEM,
                TOKEN_TYPE_CLOSE_GROUP,
            ]
            allowed_token_types_with_implicit_op = {
                TOKEN_TYPE_OPEN_GROUP: '9',
                TOKEN_TYPE_LITERAL: '9',
            }
            
        # after literal value
        if last_token_type in [
            TOKEN_TYPE_LITERAL,
        ]:
            allowed_token_types = [
                TOKEN_TYPE_UNARY_RIGHT,
                TOKEN_TYPE_BINARY,
                TOKEN_TYPE_NEW_ITEM,
                TOKEN_TYPE_CLOSE_GROUP,
            ]
            allowed_token_types_with_implicit_op = {
                TOKEN_TYPE_OPEN_GROUP: '9',
                TOKEN_TYPE_LITERAL: '9',
            }

        # after operator
        if last_token_type in [
            TOKEN_TYPE_BINARY,
            TOKEN_TYPE_UNARY_LEFT,

        ]:
            allowed_token_types = [
                TOKEN_TYPE_OPEN_GROUP,
                TOKEN_TYPE_UNARY_LEFT,
                *TOKEN_TYPE_VALUE,
            ]
            
        
        # find matching token type
        token_type, token_str = None, None
        
        all_allowed_token_types = [
            list(allowed_token_types_with_ans.items()),
            [(t, None) for t in allowed_token_types],
            list(allowed_token_types_with_implicit_op.items()),
        ]
        all_allowed_token_types = sum(all_allowed_token_types, [])
            
        for possible_token_type, implicit_op in all_allowed_token_types:
            re_pattern = re_tokens[possible_token_type]
            
            l = re_match_length(string[i:], re_pattern)
            if l > 0:
                
                if implicit_op == '':
                    tokens.append((TOKEN_TYPE_LITERAL, implicit_op))
                elif implicit_op != None:
                    tokens.append((TOKEN_TYPE_BINARY, implicit_op))
                
                token_type = possible_token_type
                token_str = string[i:i+l]
                break
                

                    
        # invalid token
        if token_type == None:
            raise Exception(f"Token not allowed at position {i}")
            i += 1
        else:
            tokens.append((token_type, token_str))
            i += len(token_str)

    return tokens


## Treeify

In [ ]:
def bubble_up(focus, new):
    """
    Finds ancestor/parent in tree upwards from `token` thats the first group token or the first
    """
    
    while True:
#         print('bubble_up', focus.value, type(focus))
#         if type(focus) != NodeValue and (type(focus) == NodeGroup or focus.precedence < new.precedence):
        if type(focus) == NodeGroup or focus.precedence < new.precedence:
#             print('return', focus.value)
            return focus
        else:
            focus = focus.parent

In [ ]:
def bubble_up_to_group(focus):
    while True:
        if type(focus) == NodeGroup:
            return focus
        else:
            focus = focus.parent

In [59]:
# TOKEN_TYPE_STRING
# TOKEN_TYPE_INTEGER
# TOKEN_TYPE_NUMBER
# TOKEN_TYPE_LITERAL

# TOKEN_TYPE_OPEN_GROUP
# TOKEN_TYPE_BINARY
# TOKEN_TYPE_UNARY_LEFT
# TOKEN_TYPE_UNARY_RIGHT

# TOKEN_TYPE_NEW_ITEM
# TOKEN_TYPE_CLOSE_GROUP

def build_token_tree(tokens):
    
    root = NodeGroup(find_token_definition('{', TOKEN_TYPE_OPEN_GROUP, lambda x:x.open_token), None)
    focus = root
    
    for token_type, value in tokens:
        
        
        # focus is
        # Binary Operator
        # Left Unary Operator
        if type(focus) in [
            NodeBinary,
            NodeUnaryLeft,
        ]:
            
            # next is 
            # Unary left
            if token_type == TOKEN_TYPE_UNARY_LEFT:
#                 print('insert lef unary')
                
                operand = find_token_definition(value, TOKEN_TYPE_UNARY_LEFT)
                next_node = NodeUnaryLeft(operand)
                
                focus.set_right(next_node)
                focus = next_node
                
            # next is
            # Group
            elif token_type == TOKEN_TYPE_OPEN_GROUP:
#                 print('insert open group')
                
                group = find_token_definition(value, TOKEN_TYPE_OPEN_GROUP, lambda x:x.open_token)
                next_node = NodeGroup(group)
                focus.set_right(next_node)
                focus = next_node
            
            # next is
            # TOKEN_TYPE_STRING
            # TOKEN_TYPE_INTEGER
            # TOKEN_TYPE_NUMBER
            # TOKEN_TYPE_LITERAL
            elif token_type in TOKEN_TYPE_VALUE:
#                 print('insert value')
                
                next_node = NodeValue(value, token_type)
                focus.set_right(next_node)
                focus = next_node
                
            else:
                raise Exception(f"token '{value}' not allowed here")
            
            
        # focus is
        # Value
        # Closed Group
        elif type(focus) == NodeValue or (type(focus) == NodeGroup and focus.is_complete()):
            
            # next is
            # Binary
            if token_type == TOKEN_TYPE_BINARY:
#                 print('insert binary')
                
                operand = find_token_definition(value, TOKEN_TYPE_BINARY)
                next_node = NodeBinary(operand)
                    
                parent_node = bubble_up(focus.parent, next_node)
                child_node = parent_node.get_right()
                parent_node.set_right(next_node)
                next_node.set_left(child_node)
                focus = next_node
                
            
            # next is
            # Unary right
            elif token_type == TOKEN_TYPE_UNARY_RIGHT:
#                 print('insert right unary')
                
                operand = find_token_definition(value, TOKEN_TYPE_UNARY_RIGHT)
                next_node = NodeUnaryRight(operand)
                    
                parent_node = focus.parent # bubble_up(focus.parent, next_node)
                child_node = parent_node.get_right()
                parent_node.set_right(next_node)
                next_node.set_left(child_node)
                # focus = next_node
            
            # next is
            # New item
            elif token_type == TOKEN_TYPE_NEW_ITEM:
#                 print('new item')
                
                token_definition = find_token_definition(value, TOKEN_TYPE_NEW_ITEM)
    
                parent_node = bubble_up_to_group(focus.parent)
                
                if token_definition.token in parent_node.group_definition.seperators:
                    
                    depth = parent_node.group_definition.seperators[token_definition.token]
                    if depth > 0:
                        parent_node.increase(depth)
                        focus = parent_node
                else:
                    raise Exception(f"token '{value}' not allowed here")
                    
            
            # next is
            # Close group
            elif token_type == TOKEN_TYPE_CLOSE_GROUP:
#                 print('close group')
                
                parent_node = bubble_up_to_group(focus.parent)
                parent_node.close()
                focus = parent_node
            
            # next is
            else:
                raise Exception(f"token '{value}' not allowed here")
#                 print('start new item in parent group')
        
        
        # focus is
        # Open Group
        elif type(focus) == NodeGroup and not focus.is_complete(): 
            
            # next is
            # Binary - use ans on left
            if token_type == TOKEN_TYPE_BINARY:
#                 print('insert binary with ans as left')

                left = NodeValue('ans', TOKEN_TYPE_LITERAL)
                
                operand = find_token_definition(value, TOKEN_TYPE_BINARY)
                next_node = NodeBinary(operand)
                next_node.set_left(left)
                
                focus.add(next_node)
                focus = next_node
            
            # next is
            # Unary left
            elif token_type == TOKEN_TYPE_UNARY_LEFT:
#                 print('add unary left')
                
                operand = find_token_definition(value, TOKEN_TYPE_UNARY_LEFT)
                next_node = NodeUnaryLeft(operand)
                
                focus.add(next_node)
                focus = next_node
            
            # next is
            # Group
            elif token_type == TOKEN_TYPE_OPEN_GROUP:
#                 print('add open group')
                
                group = find_token_definition(value, TOKEN_TYPE_OPEN_GROUP, lambda x:x.open_token)
                next_node = NodeGroup(group)
                focus.add(next_node)
                focus = next_node
            
            # next is
            # Close Group
            elif token_type == TOKEN_TYPE_CLOSE_GROUP:
#                 print('close group')
                
                focus.close()
            
            # next is
            # TOKEN_TYPE_STRING
            # TOKEN_TYPE_INTEGER
            # TOKEN_TYPE_NUMBER
            # TOKEN_TYPE_LITERAL
            elif token_type in TOKEN_TYPE_VALUE:
#                 print('add value')
                
                next_node = NodeValue(value, token_type)
                focus.add(next_node)
                focus = next_node
                
            # next is 
            # TOKEN_TYPE_NEW_ITEM
            elif token_type == TOKEN_TYPE_NEW_ITEM:
#                 print('new item/dimension')
                
                token_definition = find_token_definition(value, TOKEN_TYPE_NEW_ITEM)
                
                if token_definition.token in focus.group_definition.seperators:
                    
                    depth = focus.group_definition.seperators[token_definition.token]
                    if depth > 0:
                        focus.increase(depth)
                else:
                    raise Exception(f"token '{value}' not allowed here")
                
            # next is
            else:
                raise Exception(f"token '{value}' not allowed here")
#                 print('start new item in parent group')
            
        
    
    return root
    

## Computation Graph

In [60]:
def build_computation_graph(token_tree):
    return token_tree.get_evaluable()

# Excecute

## Environment

In [61]:
class Environment():
       
    def __init__(self, parent=None, dictionary=None):
        self.parent = parent
        self.dictionary = dictionary or {}
        self.ln_count = 1
        self.ans_available = False
        
    def __getitem__(self, key):
        
        if key in self.dictionary:
            return self.dictionary[key]
        elif self.parent != None:
            return self.parent[key]
        else:
            assert False, f'key {key} is not defined in this scope'
        
    def __setitem__(self, key, value):
        
        if type(key) in [list, tuple] and type(value) in [list, tuple]:
            for k,v in zip(key, value):
                self[key] = value
        
        else:
            if key in self.dictionary:
                self.dictionary[key] = value
            elif self.parent != None:
                self.parent[key] = value
            else:
                self.dictionary[key] = value
    
    def __contains__(self, key):
        
        if key in self.dictionary:
            return True
        elif self.parent != None:
            return key in self.parent
        else:
            return False
        
    def addLn(self, ln):
        name = f'ln{self.ln_count}'
        self[name] = ln
        self[''] = ln
        
        self.ln_count += 1
        self.ans_available = True
        
        return name
    
    def enterScope(self, dictionary=None):
        return Environment(self, dictionary)
        
    def exitScope(self):
        return self if self.parent == None else self.parent
        
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'Environment({self.dictionary, self.parent})'

In [62]:
def create_base_environment():
    built_in_variables = {k:v for k,v in built_ins.vars}
    built_in_functions = {}
    built_in_conversion_function_sets = {}
    
    for name, function, parameter_names, parameter_types, parameter_ranks, parameter_shapes in built_ins.functions:
        
        parameter_evaluation_parameters = built_ins.function_parameter_evaluation_parameters.get(name, {})
        
        built_in_functions[name] = built_in_functions.get(name, FunctionSet(name, parameter_evaluation_parameters))
        built_in_functions[name].add(FunctionSignature(
            name, 
            function, 
            parameter_names, 
            parameter_types, 
            parameter_ranks, 
            parameter_shapes
        ))
        
    for name, forward, reverse in built_ins.conversion_function_sets:
        built_in_conversion_function_sets[name] = ConversionFunctionSet(name, forward, reverse)
    
    environment = Environment(None, {**built_in_variables, **built_in_functions, **built_in_conversion_function_sets})
    
    return environment

## Evaluate

In [63]:
def evaluate(computation_graph, environment):
    return computation_graph.eval(environment)

## Run

In [64]:
def print_result(result, ln_name):
    ln_name = ln_name + ': '
    space = ' ' * len(ln_name)
    lines = str(result).split('\n')
    pre = [ln_name] + (len(lines)-1) * [space]
    
    lines = [p+l for p,l in zip(pre, lines)]
    
    print(*lines, sep='\n')

In [65]:
def calc(query, environment, debug=False):
        
#     commands
#     if   query == 'exit':   break
#     elif query == 'help':   help()
#     elif query == 'ref':    ref()
#     elif query == 'clear':  clear()
#     elif query == 'copy':   pyperclip.copy(ans)
#     elif query == '=':      pyperclip.copy(ans)

#     # evaluate query
#     elif query != "":
    

#     if debug: print('environment', environment, end='\n\n')
    
    tokens = lexing(query, environment.ans_available)
    if debug: print('tokens', tokens, end='\n\n')
    
    token_tree = build_token_tree(tokens)
    if debug: print('token_tree', token_tree, end='\n\n')
    
    computation_graph = build_computation_graph(token_tree)
    if debug: print('computation_graph', computation_graph, end='\n\n')
    
    result = evaluate(computation_graph, environment)
    
    ln_name = environment.addLn(result)
    print_result(result, ln_name)

    return result

## Test

### Basics

In [66]:
e = create_base_environment()

In [67]:
calc('0.1 + 0.2', e)

ln1: 0.3


Tensor([Real(0.3)])

In [68]:
calc('[phi, 1/phi]', e)

ln2: [1.61803398874989484820458683436563811772, 0.6180339887498948482045868344]


Tensor([Real(1.61803398874989484820458683436563811772)
 Real(0.6180339887498948482045868344)], (2,))

In [69]:
calc('(1 + [1,2,3]) * 2; 1 + [1,2,3] * 2', e)

ln3: [3, 5, 7]


Tensor([Real(3) Real(5) Real(7)], (3,))

In [70]:
calc('2 ^ [1,2,3,4]; [1,2,3,4] ^ 2', e)

ln4: [1, 4, 9, 16]


Tensor([Real(1) Real(4) Real(9) Real(16)], (4,))

In [71]:
calc('[2,4,8] ^ [2,3,4]', e)

ln5: [4, 64, 4096]


Tensor([Real(4) Real(64) Real(4096)], (3,))

In [72]:
calc('PI * [1/2, 1, 2,,,]', e)

ln6: [[[1.570796326794896619231321692, 3.141592653589793238462643383, 6.283185307179586476925286767]]]


Tensor([Real(1.570796326794896619231321692) Real(3.141592653589793238462643383)
 Real(6.283185307179586476925286767)], (1, 1, 3))

In [73]:
calc('[1,2,,3,4,,5,6] # [1,2,3,,4,5,6]', e)

ln7: [[9, 12, 15],
      [19, 26, 33],
      [29, 40, 51]]


Tensor([Real(9) Real(12) Real(15) Real(19) Real(26) Real(33) Real(29) Real(40)
 Real(51)], (3, 3))

In [74]:
calc('|0 - |10-100| |; |0-1|; ||0-1||; |[1,-1,2,-2]|', e)

ln8: [1, 1, 2, 2]


Tensor([Real(1) Real(1) Real(2) Real(2)], (4,))

In [75]:
calc('$[1,4,9,16]; 3$[1,8,27,64]; 0.1$10; 10^(1/0.1)', e)

ln9: 10000000000


Tensor([Real(10000000000)])

In [76]:
calc('[1,2,3] .* [3,2,1]; [1,2,3,,] # [3,,2,,1]; [1,2,3,,4,5,6] .* [4,5,6,,7,8,9]', e)

ln10: [32, 122]


Tensor([Real(32) Real(122)], (2,))

In [77]:
calc("""
[
    1,2,3,,
    4,5,6
    ,,,
    6,5,4,,
    3,2,1
    ,,,
    1,2,3,,
    4,5,6
]
""", e)

ln11: [[[1, 2, 3],
        [4, 5, 6]],
      
       [[6, 5, 4],
        [3, 2, 1]],
      
       [[1, 2, 3],
        [4, 5, 6]]]


Tensor([Real(1) Real(2) Real(3) Real(4) Real(5) Real(6) Real(6) Real(5) Real(4)
 Real(3) Real(2) Real(1) Real(1) Real(2) Real(3) Real(4) Real(5) Real(6)], (3, 2, 3))

In [78]:
calc("""
[[[1, 2, 3],
  [4, 5, 6]],

 [[6, 5, 4],
  [3, 2, 1]],

 [[1, 2, 3],
  [4, 5, 6]]]
""", e)

ln12: [[[1, 2, 3],
        [4, 5, 6]],
      
       [[6, 5, 4],
        [3, 2, 1]],
      
       [[1, 2, 3],
        [4, 5, 6]]]


Tensor([Real(1) Real(2) Real(3) Real(4) Real(5) Real(6) Real(6) Real(5) Real(4)
 Real(3) Real(2) Real(1) Real(1) Real(2) Real(3) Real(4) Real(5) Real(6)], (3, 2, 3))

### Variables

In [79]:
calc('x = 1; y = 2; x = 3; x^y', e)

ln13: 9


Tensor([Real(9)])

In [80]:
calc('a = ((2, 4), 8); ((x, y), z) = a; x^y; y^z', e)

ln14: 65536


Tensor([Real(65536)])

In [81]:
calc('x = [1,2,,3,4]; (a,b,c,d) = x; (a,b,c,d)', e)

ln15: (1, 2, 3, 4)


Array([Tensor([Real(1)]) Tensor([Real(2)]) Tensor([Real(3)]) Tensor([Real(4)])])

### Implicit Multiplication

In [82]:
# print(calc('x = 5; 2 * x; 2(3)(3+4)x; 3x^2'))

In [83]:
# print(calc('x = 5; 2 x x'))

In [84]:
# print(calc('f = x => 2x; 3 f(5)'))

### Functions

In [85]:
calc('fun(2, 4); fun(3); fun([1,2;3,4], [5,6;7,8])', e)

ln16: [[5, 12],
       [21, 32]]


Tensor([Real(5) Real(12) Real(21) Real(32)], (2, 2))

In [86]:
calc('f = x => x^3; f(5)', e)

ln17: 125


Tensor([Real(125)])

In [87]:
calc('f = (x, y) => x^y; f(3,4); f([1,2,3], [4,5,6])', e)

ln18: [1, 32, 729]


Tensor([Real(1) Real(32) Real(729)], (3,))

In [88]:
calc('f = x => {y = x + 1; x^y}; f(3)', e)

ln19: 81


Tensor([Real(81)])

In [89]:
calc("""
    f = x => {
        inc = x => x + 1
        y = inc(x)
        x^y
    }
    g = x => x^x
    
    (
        f(2)
        g(2)
    )
""", e)

ln20: (8, 4)


Array([Tensor([Real(8)]) Tensor([Real(4)])])

### Array and Tensor Item Lookup

In [90]:
calc("""
    f = x => x^2
    c = 1
    a = (1,2,3)
    t = [0.1, 0.2, 0.3,, 0.4, 0.5, 0.6]
    t(1,c)
    a(c)
""", e)

ln21: 2


Tensor([Real(2)])

In [91]:
calc('a = (1,2,3); a(1)', e)

ln22: 2


Tensor([Real(2)])

In [92]:
calc('t = [1,2,3]; t(1)', e)

ln23: 2


Tensor([Real(2)])

### Conversions

In [93]:
calc('120cm + 15mm @ mm', e)

ln24: 1215.000


Tensor([Real(1215.000)])

In [94]:
calc('1m * 1cm @ mm2', e)

ln25: 10000.00


Tensor([Real(10000.00)])

In [95]:
calc('[1,2,3]cm', e)

ln26: [0.01, 0.02, 0.03]


Tensor([Real(0.01) Real(0.02) Real(0.03)], (3,))

In [96]:
calc('[1mm, 1cm, 1m, 1km] @ m', e)

ln27: [0.001, 0.01, 1, 1000]


Tensor([Real(0.001) Real(0.01) Real(1) Real(1000)], (4,))

In [97]:
calc('160cm * 50cm * 40cm @ l', e)

ln28: 320.000


Tensor([Real(320.000)])

In [98]:
calc('1hr + 5min + 1000ms @ time', e)

ln29: "01h:05m:01s"


Tensor([String("01h:05m:01s")])

In [99]:
calc("'1d 2:40:0.0001'time + '3d 1:35:0.001'time @ time", e)

ln30: "4d 04h:15m:00.001100s"


Tensor([String("4d 04h:15m:00.001100s")])

In [100]:
calc("'1:35'time * 3 @ time", e)

ln31: "04h:45m:00s"


Tensor([String("04h:45m:00s")])

In [101]:
calc("('10:00'time * 5) % 24hr @ time", e)

ln32: "02h:00m:00s"


Tensor([String("02h:00m:00s")])

In [102]:
calc('"1h:2m:3s"time * 30 @ time', e)

ln33: "1d 07h:01m:30s"


Tensor([String("1d 07h:01m:30s")])

In [103]:
calc('x = 5cm @ mm; x', e)

ln34: 0.05


Tensor([Real(0.05)])

### Continued Equations

In [104]:
calc('3 ^ 4', e)
calc(' / 4', e)

ln35: 81
ln36: 20.25


Tensor([Real(20.25)])

In [105]:
calc('2 ^ 3', e)
calc('ln37 * 2', e)

ln37: 8
ln38: 16


Tensor([Real(16)])

### [NEXT]